In [1]:
import networkx as nx
import math
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from networkx.algorithms import bipartite
import sys
import contextlib
import networkx as nx
import pandas as pd
from ipynb.fs.defs.functions import *

## Simulation of empirical networks
This notebook outlines how to numerically simulate the fixation probability of a weighted empirical network. We demonstrate the process using the example of the ants colony weighted networks. The other empirical networks were similarly evaluated.

In [2]:


# Load the DataFrame
ants = pd.read_csv("insecta-ant-trophallaxis-colony2.csv")

# Create an undirected graph
G = nx.Graph()

# Iterate through the DataFrame to update the network
for index, row in ants.iterrows():
    node1 = int(row['n1'])
    node2 = int(row['n2'])
    
    # Check if the edge already exists
    if G.has_edge(node1, node2):
        # If the edge exists, increment the weight by one
        G[node1][node2]['weight'] += 1
    else:
        # If the edge does not exist, add it with a weight of one
        G.add_edge(node1, node2, weight=1)

# Now G contains the undirected weighted network
mapping = {39:0}
Gprime = nx.relabel_nodes(G, mapping)

In [ ]:
#Simulating the empirical network of ants
import time
from timeit import default_timer as timer
from multiprocessing import Pool, cpu_count

def main(j):
    
    start = timer()
    arg=[]
    c=[]

    print(f'starting computations on {cpu_count()} cores')
    
    for i in range(0,cpu_count()):
        arg.append((Gprime,350000,j))


    with Pool() as pool:
        c.append(pool.starmap(weighted_simulation_count, arg))
        

    end = timer()
    print(f'elapsed time: {end - start}')
    return (sum(c[0])/len(c[0]))

if __name__ == '__main__':
    c=[]
    d_weighted_ants_39_nodes_new=[]
    for k in range(7,17):
        d_weighted_ants_39_nodes_new.append(main(k/10))
        np.save('d_weighted_ants_39_nodes_new.npy' , np.array(d_weighted_ants_39_nodes_new))

starting computations on 40 cores


# Code

In [9]:
import networkx as nx
import math
import numpy as np
import random
import matplotlib.pyplot as plt
from networkx.algorithms import bipartite
import sys
import contextlib
import pandas as pd

def unicolor_all_nodes(graph):
    for i in range (0,graph.number_of_nodes()):
        graph.nodes[i]['color']='red'
#reset button for S        
def reset_button(graph):
    for i in range (0,graph.number_of_nodes()):
        graph.nodes[i]['color']='red'
        
def theoretic_fixation_moran(nodes,fitness): 
    k=(1-1/fitness)/(1-1/(pow(fitness,nodes)))
    return k

#Main Chunk,
def simulate(graph,fitness):
    unicolor_all_nodes(graph)
    #A = nx.adjacency_matrix(graph)
    
    
    #random mutation
    graph.nodes[np.random.randint(0,graph.number_of_nodes())]['color']='blue'
    #print(graph.nodes.data())
    #Array with colors of the nodes of the graph,
    Color_Array = np.empty(graph.number_of_nodes(), dtype='object')  
    
    #Color_array updates it's colors according to mutation
    for i in range(0,graph.number_of_nodes()):
        Color_Array[i]=graph.nodes[i]['color']    
    
    #init variables
    red=0
    redprob=0
    contenderlist=[]
    contender=0
    victimlist=[]
    victim=0
    
    #loop runs until all nodes are same colored
    while len(np.unique(Color_Array)) > 1:
        
        contenderlist=[]
        victimlist=[]
        red=0
        
        for i in range(0,graph.number_of_nodes()):
            if graph.nodes[i]['color']=='red':
                red = red + 1
        #print(red," resident remain")
        
        redprob = (red)/(fitness*(graph.number_of_nodes()-red)+red)
        
        ranBirth = random.uniform(0, 1)
        
        if ranBirth <= redprob :
            for i in range(0,graph.number_of_nodes()):
                if graph.nodes[i]['color']=='red':
                    contenderlist.append(i)
        else:
            for i in range(0,graph.number_of_nodes()):
                if graph.nodes[i]['color']=='blue':
                    contenderlist.append(i)
                    
        contender=random.choice(contenderlist)
        
        #now we select the victim amongst the neighbours of the contender
        
        for i in range(0,graph.number_of_nodes()):
            if int(i in graph.neighbors(contender)) == 1:
                victimlist.append(i)
        
        victim = random.choice(victimlist)
        graph.nodes[victim]['color'] = graph.nodes[contender]['color']
        
        Color_Array[victim]= graph.nodes[contender]['color']
        #print(Color_Array)
        
        
        #checking who survived
        #if len(np.unique(Color_Array)) == 1:
            #print(Color_Array[0], "population survived")
        
        


        
        
        
        
        
        
        
def simulation_count(graph,n,r):
    
    blue_victory=0
    red_victory=0
        
        
    for i in range(n):
        simulate(graph,r)

            
        if graph.nodes[1]['color']=='red':
                red_victory=red_victory+1
        elif graph.nodes[1]['color']=='blue':
                blue_victory=blue_victory+1
                    
        #return the fixation prob
    prob = blue_victory/(n)
    return prob
        
        #print(r, " , ",blue_victory/(n))
        #print(' {:12.4e} {:12.4e}'.format(r, blue_victory/(n)))
                   
        #print("residents won",red_victory,"times and mutants won",blue_victory,"times")
        #print("Computational fixation probability for",graph," with fitness ",r, " is",blue_victory/(n))
        #array.append(blue_victory/(n)) 
        #should it be n+1 ?
        




def part_simulate(graph,fitness,time):
    n=0
    while len(np.unique([graph.nodes[i]['color'] for i in range(0,graph.number_of_nodes())])) > 1 and n<time:
        #print("graph1",n)
        #print(c)
        contenderlist=[]
            
        n=n+1
        #c=c+1
        
        red=[graph.nodes[i]['color'] for i in range(0,graph.number_of_nodes())].count('red')
        redprob = (red)/(fitness*(graph.number_of_nodes()-red)+red)
            #print(redprob," for graph1")  
        ranBirth = random.uniform(0, 1)
        if ranBirth <= redprob :
            for i in range(0,graph.number_of_nodes()):
                if graph.nodes[i]['color']=='red':
                    contenderlist.append(i)
        else:
            for i in range(0,graph.number_of_nodes()):
                if graph.nodes[i]['color']=='blue':
                    contenderlist.append(i)
        contender=random.choice(contenderlist)
        #now we select the victim amongst the neighbours of the contender
        #first check if node is isolated, required for temporal networks
        nghbr_list=[n for n in nx.all_neighbors(graph,contender)]
        if len(nghbr_list)==0:
            continue
        victim = random.choice(nghbr_list)
        #killing the victim 
        graph.nodes[victim]['color'] = graph.nodes[contender]['color']

        #carray[victim]= graph.nodes[contender]['color']
            
            #print(Color_Array)
            
            
            
            
            
            
            
            
            
            
            
            
            
            
            
            
            
            
            
            
def weighted_part_simulate(graph,fitness,time):
    n=0
    while len(np.unique([graph.nodes[i]['color'] for i in range(0,graph.number_of_nodes())])) > 1 and n<time:
        #print("graph1",n)
        #print(c)
        contenderlist=[]
            
        n=n+1
        #c=c+1
        
        red=[graph.nodes[i]['color'] for i in range(0,graph.number_of_nodes())].count('red')
        redprob = (red)/(fitness*(graph.number_of_nodes()-red)+red)
            #print(redprob," for graph1")  
        ranBirth = random.uniform(0, 1)
        if ranBirth <= redprob :
            for i in range(0,graph.number_of_nodes()):
                if graph.nodes[i]['color']=='red':
                    contenderlist.append(i)
        else:
            for i in range(0,graph.number_of_nodes()):
                if graph.nodes[i]['color']=='blue':
                    contenderlist.append(i)
        contender=random.choice(contenderlist)
        #now we select the victim amongst the neighbours of the contender
        #first check if node is isolated, required for temporal networks
        nghbr_list=[n for n in nx.all_neighbors(graph,contender)]
        if len(nghbr_list)==0:
            continue
        weights_nbr=[]
        for nghbr in nghbr_list:
            weights_nbr.append(graph[contender][nghbr]["weight"])
        victim = random.choices(nghbr_list, weights=weights_nbr)[0]
        #killing the victim 
        graph.nodes[victim]['color'] = graph.nodes[contender]['color']

        #carray[victim]= graph.nodes[contender]['color']
            
            #print(Color_Array)
            
            
            
            
            
            
            
            
            
            
            
 
        
        
        
        
        
        
        
        
        
 
    
#Main Chunk,
def weighted_simulate(graph,fitness):
    unicolor_all_nodes(graph)
    #A = nx.adjacency_matrix(graph)
    
    
    #random mutation
    graph.nodes[np.random.randint(0,graph.number_of_nodes())]['color']='blue'
    #print(graph.nodes.data())
    #Array with colors of the nodes of the graph,
    Color_Array = np.empty(graph.number_of_nodes(), dtype='object')  
    
    #Color_array updates it's colors according to mutation
    for i in range(0,graph.number_of_nodes()):
        Color_Array[i]=graph.nodes[i]['color']    
    
    #init variables
    red=0
    redprob=0
    contenderlist=[]
    contender=0
    victimlist=[]
    victim=0
    
    #loop runs until all nodes are same colored
    while len(np.unique(Color_Array)) > 1:
        
        contenderlist=[]
        victimlist=[]
        red=0
        
        for i in range(0,graph.number_of_nodes()):
            if graph.nodes[i]['color']=='red':
                red = red + 1
        #print(red," resident remain")
        
        redprob = (red)/(fitness*(graph.number_of_nodes()-red)+red)
        
        ranBirth = random.uniform(0, 1)
        
        if ranBirth <= redprob :
            for i in range(0,graph.number_of_nodes()):
                if graph.nodes[i]['color']=='red':
                    contenderlist.append(i)
        else:
            for i in range(0,graph.number_of_nodes()):
                if graph.nodes[i]['color']=='blue':
                    contenderlist.append(i)
                    
        contender=random.choice(contenderlist)
        
        #now we select the victim amongst the neighbours of the contender
        
        nghbr_list=[n for n in nx.all_neighbors(graph,contender)]
        if len(nghbr_list)==0:
            continue
        weights_nbr=[]
        for nghbr in nghbr_list:
            weights_nbr.append(graph[contender][nghbr]["weight"])
        victim = random.choices(nghbr_list, weights=weights_nbr)[0]
       
        graph.nodes[victim]['color'] = graph.nodes[contender]['color']
        
        Color_Array[victim]= graph.nodes[contender]['color']
        #print(Color_Array)
        
        
        #checking who survived
        #if len(np.unique(Color_Array)) == 1:
            #print(Color_Array[0], "population survived")
            
            
            

        
def weighted_simulation_count(graph,n,r):
    
    blue_victory=0
    red_victory=0
        
        
    for i in range(n):
        weighted_simulate(graph,r)

            
        if graph.nodes[1]['color']=='red':
                red_victory=red_victory+1
        elif graph.nodes[1]['color']=='blue':
                blue_victory=blue_victory+1
                    
        #return the fixation prob
    prob = blue_victory/(n)
    return prob
        
        